In [3]:
import torch
from torch.nn import functional as F
from torch import nn

block_size = 64
batch_size = 32
n_heads = 2
n_layers = 6
d_model = 64 * n_heads
dropout = 0.2
learning_rate = 3e-4
epochs = 5_000
eval_iters = 200
dff = d_model * 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == 'cuda':
    torch.backends.cudnn.benchmark = True


class MultiHeadAttention(nn.Module):

    def __init__(self, n_heads, d_model, block_size, dropout=0.1):

        super().__init__()
        assert d_model % n_heads == 0, "Embedding dimension must be 0 modulo number of heads."

        self.dropout = nn.Dropout(dropout)
        self.query = nn.Linear(d_model, d_model, bias=False)
        self.key = nn.Linear(d_model, d_model, bias=False)
        self.value = nn.Linear(d_model, d_model, bias=False)
        self.att_proj = nn.Linear(d_model, d_model, bias=False)
        self.register_buffer('mask', torch.triu(torch.ones(block_size, block_size).bool(), diagonal=1))

    def forward(self, x):

        q = x
        k = x
        v = x
        B,T,_ = x.shape 
        dk = d_model // n_heads

        # linear projections
        q = self.query(q) 
        k = self.key(k) 
        v = self.value(v) 

        # add number of heads
        q = q.view(B,T,n_heads,dk).permute(0,2,1,3)   # B,T,h,dk
        k = k.view(B,T,n_heads,dk).permute(0,2,1,3)  
        v = v.view(B,T,n_heads,dk).permute(0,2,1,3)  
        
        # attention 
        x = q @ k.transpose(-2,-1) # B,h,T,dk @ B,h,dk,T --> B,h,T,T
        x = x * dk ** -0.5 # B,h,T,T
        x = x.masked_fill(self.mask, float('-inf')) # B,h,T,T
        x = F.softmax(x, dim=(-1)) # B,n_h,T,T 
        x = x @ v  # B,h,T,T @ B,T,h,dv --> B,h,T,dv
        B,h,T,dv = x.shape
        x = x.transpose(2,1).contiguous().view(B,T,h*dv) #B,T,C
        out = self.att_proj(x) # B,T,C

        return out
    
class AttentionLayer(nn.Module):
    def __init__(self, n_heads, d_model, block_size, dropout):
        super().__init__()

        self.att = MultiHeadAttention(n_heads, d_model, block_size, dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.att(x)
        return x
    
class FeedForward(nn.Module):
    def __init__(self, d_model, dff, dropout, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.seq = nn.Sequential(
                    nn.Linear(d_model, dff),
                    nn.ReLU(),
                    nn.Dropout(dropout),
                    nn.Linear(dff, d_model)
                    )

    def forward(self, x):
        x = self.seq(x)
        return x
    
class DecoderLayer(nn.Module):
    def __init__(self, n_heads, d_model, block_size, dropout, dff) :
        super().__init__()

        self.att = AttentionLayer(n_heads, d_model,
                                  block_size, dropout)
        
        self.ffw = FeedForward(d_model, dff, dropout)
        self.lnorm1 = nn.LayerNorm(d_model)
        self.lnorm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):

        x = x + self.att(self.lnorm1(x))
        x = x + self.ffw(self.lnorm2(x))

        return x

In [4]:
class Model(nn.Module):

    def __init__(self, vocab_size, block_size, dropout, dff, n_layers, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.block_size = block_size
        self.embedding_table = nn.Embedding(vocab_size, d_model)
        self.pos_embedding = nn.Embedding(block_size, d_model)

        self.decoder = nn.Sequential(*[DecoderLayer(n_heads,
                                                    d_model,
                                                    block_size,
                                                    dropout,
                                                    dff) 
                                                    for _ in range(n_layers)])

        self.out = nn.Linear(d_model, vocab_size)

    def forward(self, x, targets=None):

        embeds = self.embedding_table(x)
        positions = self.pos_embedding(torch.arange(block_size, device=device))
        x = embeds + positions
        x = self.decoder(x)
        logits = self.out(x)

        if targets == None:
            loss = None

        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(input=logits, target=targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        B, T = idx.shape
        if T < self.block_size:
            # pad the input with zeros if it's less than block_size
            idx = F.pad(idx, (0, self.block_size - T))
        for _ in range(max_new_tokens):
            # use only the last block_size tokens
            idx_cond = idx[:, -self.block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
data = open('text.txt').read()
vocab = list(sorted(set(data)))
vocab_size = len(vocab)

stoi = {c:i for i, c in enumerate(vocab)}
itos = {i:c for i, c in enumerate(vocab)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda i: ''.join([itos[i] for i in i])

data = torch.tensor(encode(data))

n_tr = int(len(data) * 0.9)
n_val = len(data) - n_tr

train = data[:n_tr]
val = data[n_tr+1:]

def make_batches(split):

    data = train if split == 'train' else val
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    x, y = x.to(device), y.to(device)
    
    return x, y

Xb, Yb = make_batches('train')
m = Model(vocab_size, block_size, dropout, dff, n_layers).to(device)
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)
n_params = sum(p.nelement() for p in m.parameters())
print(f'Number of parameters: {n_params}')

@torch.no_grad()
def estimate_loss(m):
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = make_batches(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

for epoch in range(epochs):

    Xb, Yb = make_batches('train')
    logits, loss = m(Xb, Yb) # B, C

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 999:
        l = estimate_loss(m)
        print(f"Iteration {epoch}. Training Loss: {l['train']:.3f}. Evaluation Loss: {l['val']:.3f}")

max_new_tokens = 2_000
seed_text = ". "
seed_idx = torch.tensor([stoi[c] for c in seed_text], device=device).unsqueeze(0)
predictions = m.generate(seed_idx, max_new_tokens).to(device)
pad_len = m.block_size
generated_text = decode(predictions[0].tolist())
generated_text = generated_text[pad_len:]  # Remove leading padding
print(generated_text)

Number of parameters: 1211457
Iteration 999. Training Loss: 1.944. Evaluation Loss: 2.029
Iteration 1999. Training Loss: 1.712. Evaluation Loss: 1.863
Iteration 2999. Training Loss: 1.604. Evaluation Loss: 1.769
Iteration 3999. Training Loss: 1.542. Evaluation Loss: 1.715
Iteration 4999. Training Loss: 1.492. Evaluation Loss: 1.679
PARETIO:
That, you, father nonce.

YORK:
Anongely bear,
In here smore than a if click you
My nevet so house.

CLARENCE:
Thought nothan More, how look,
But it in things, seemble better tomee.

CORIOLANUS:
Humard.

MONTES:
He repect of this alkemand;
For the Simplers wry that has enester;
Antea, for your wince bretts and fo
winger this could sie: at thyself:
We he hone die pleasu.

FLORW:
Hashow so, letter to you.

LUCIO:
Beut, no highnot, he honow you?

Both:
He mede of this genered, where prinood,
Or art madosice Come, but, the pacisonantey,
But thyself the marrily, so, say, bright's now.

CLARENCE:
You so do not is my hand,
And curtabled time to help tear-t